# Test des fonctions de l'API OpenAI avec OpenFisca

Ce notebook permet de :
- Receuillir une demande de réforme exprimée en langage naturel : _Je souhaite réduire de 10% la CSG imposable pour les personnes qui travaillent._
- Demander à OpenAI de l'exprimer en terme de modification de paramètre OpenFisca, grâce aux _Functions_ d'OpenAI. Pour cela une liste de paramètres est envoyé en plus de l'entrée de l'utilisateur et d'un schéma de la réponse attendue.
- Réaliser un chiffrage de la réforme à l'aide d'une API, ici celle de LexImpact, mais cela pourrait être l'API publique d'OpenFisca sur un cas type.
- Mettre sous forme d'amendement le résultat du chiffrage en appelant OpenAI.

## Installation


Il faut créer un fichier .env avec les informations suivantes :
```
OPENAI_API_KEY=xxxx Clef OpenAI
LEX_API_URL=https://xxxx Adresse API Simu-Etat de LexImpact
LEX_TOKEN=xxxxx Token API Simu-Etat de LexImpact
```

Les dépendances sont les suivantes :

```
pip install openai python-decouple ipykernel ipywidgets
python -m ipykernel install --name llm-kernel --user
```

Lancement avec Voilà pour en faire une application Web interactive :

`voila --Voila.ip='0.0.0.0' --Voila.base_url=/voila/ --no-browser`

## Exemple de réponse

```
AMENDEMENT

A l’article [insérer le numéro de l'article], après le [indiquer le point, l'alinéa ou le paragraphe] insérer le paragraphe suivant :

« Le taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité est réduit de 5% pour atteindre 6,46%. Le taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité est réduit de 10% pour atteindre 2,16%. »

EXPOSÉ DES MOTIFS

Cet amendement vise à alléger la pression fiscale sur les revenus d'activité en réduisant le taux de la CSG déductible et le taux de la CSG imposable.

En effet, le taux de la CSG déductible sur les revenus d'activité passera de 6,8% à 6,46% et le taux de la CSG imposable sur les revenus d'activité sera réduit de 2,4% à 2,16%.

Ces mesures permettront d'augmenter le pouvoir d'achat des travailleurs et de stimuler l'économie.

Concernant l'impact sur les finances publiques, avant la réforme, le montant de la CSG déductible sur les salaires s'élevait à -64 304 294 891,98 euros et celui de la CSG imposable à -22 695 633 461,51 euros. Après l'application du Projet de Loi de Finances proposé par le gouvernement, ces montants resteraient inchangés.

Cependant, avec l'adoption de cet amendement, le montant de la CSG déductible sur les salaires serait de -61 089 080 024,71 euros et celui de la CSG imposable de -20 426 070 169,84 euros.

Il est donc nécessaire de trouver des moyens de compenser cette perte de revenus pour l'Etat, tout en soulageant la pression fiscale sur les travailleurs.

```

In [2]:
from decouple import Config, RepositoryEnv
import openai
import json
from IPython.display import Markdown
import requests
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

In [3]:
DOTENV_FILE = '.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))
openai.api_key = env_config('OPENAI_API_KEY')

In [4]:
# Définition de la fonction
generate_tax_reform = [
    {
        'name': 'generate_tax_reform',
        'description': """Generate a tax reform proposal using OpenFisca Parameters.

        """,
        'parameters': {
            'type': 'object',
            'properties': {
                'Reform description': {
                    'type': 'string',
                    'description': 'Description of the reform to generate.'
                },
                'parameters_change': {
                    
                    "type": "array",
                    "items": {
                        'type': 'object',
                        'properties':
                        {
                            'parameter_description': {
                                'type': 'string',
                                'description': 'Description of the parameter to change.'
                            },
                            'parameter_name': {
                                'type': 'string',
                                'description': 'OpenFisca parameter name.'
                            },
                            'value': {
                                'type': 'string',
                                'description': 'New value of the parameter. Must be a unique value, not a mathematical operation.'
                            },
                        },
                    }
                }
            }
        }
    }
]

In [5]:
# Préparation du prompt
# reforme_en_francais = "Je souhaite réduire de 10% la CSG imposable pour les personnes qui travail et de 5% la CSG déductible."

def get_prompt(reforme_en_francais):
    return f"""
You are an lawer at french national assembly. You are asked to reform the CSG taxes by a deputy.
You can change the following parameters in the law:
For the "Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité" the OpenFisca parameter name is "prelevements_sociaux.contributions_sociales.csg.activite.deductible.taux" and the default value is 0.068
For the "Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité" the OpenFisca parameter name is "prelevements_sociaux.contributions_sociales.csg.activite.imposable.taux" and the default value is 0.024
Here is what the deputy said:
{reforme_en_francais}
"""


In [6]:
# Appel de l'API OpenAI Functions
def call_openai_function(prompt):
    reforms = [prompt]
    for sample in reforms:
        response = openai.ChatCompletion.create(
            model = 'gpt-4',  # gpt-3.5-turbo',
            messages = [{'role': 'user', 'content': sample}],
            functions = generate_tax_reform,
            function_call = 'auto'
        )

        # Loading the response as a JSON object
        json_response = json.loads(response['choices'][0]['message']['function_call']['arguments'])
        # print(json_response)
        return json_response

In [7]:
# Appel de l'API LexImpact
def get_leximpact_payload(openai_response):
    api_payload = {
        "amendement": {},
        "base": 2024,
        "plf": 2024,
        "output_variables": [
            "csg_deductible_salaire",
            "csg_imposable_salaire"
        ],
    }
    for reform in openai_response["parameters_change"]:
        api_payload['amendement'][reform["parameter_name"]]= {
            "start": "2024-01-01",
            "type": "parameter",
            "variable": "csg_salaire",
            "value": float(reform["value"])
        }
    return api_payload

# api_payload

In [8]:
# Appel de l'API LexImpact
def call_leximpact(api_payload):
    url = env_config('LEX_API_URL') + "/state_simulation"
    headers = {'Content-Type': 'application/json','jwt-token': env_config('LEX_TOKEN')}
    api_response = requests.post(url, headers=headers, data=json.dumps(api_payload))
    return api_response
# response.json()

In [9]:
# Rédaction de l'amendement
def get_completion(prompt, model="gpt-4", temperature=0):
    """
    model : "gpt-3.5-turbo" | "gpt-4"
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [10]:
def prepare_response_prompt(openai_response, api_response):
    return f"""
Vous êtes un député à l'assemblée nationale. Vous êtes chargé de réformer la CSG.
Votre assistant informaticien vous a préparé un amendement que vous pouvez déposer à l'assemblée nationale.
Le Taux déductible de la contribution sociale généralisée (CSG) déductible sur les revenus d'activité" est actuellement de 6,8%.
Le Taux de la contribution sociale généralisée (CSG) imposable sur les revenus d'activité correspond à 2,4% du salaire brut.

Voici le contenu de l'amendement en JSON :
{openai_response}

Voici son impact sur les finances publiques qui doit être mentionné dans l'exposé des motifs de l'amendement :
- Montant en euro avant réforme : {api_response.json()['result']['base']['state_budget']}
- Le montant en euro après application du Projet de Loi de Finances proposé par le gouvernement : {api_response.json()['result']['plf']['state_budget']} 
- Le montant après votre amendement : {api_response.json()['result']['amendement']['state_budget']}

Vous devez rédiger l'amendement en français.
"""


In [11]:
text_input = widgets.Textarea(
    value='Je souhaite réduire de 10% la CSG imposable pour les personnes qui travaillent et de 5% la CSG déductible.',
    placeholder='Décrivez votre réforme',
    description='Description de votre amendement : ',
    disabled=False,
    layout={'height': '100px', 'width': '500px'}
)
reforme_en_francais = text_input.value
display(text_input)

Textarea(value='Je souhaite réduire de 10% la CSG imposable pour les personnes qui travaillent et de 5% la CSG…

In [12]:
button = widgets.Button(description="Chiffrer et rédiger")
output = widgets.Output()

display(button, output)
def on_button_clicked(b):
    with output:
        print("Conversion de la réforme en format OpenFisca...")
    reforme_en_francais = text_input.value
    prompt = get_prompt(reforme_en_francais)
    openai_response = call_openai_function(prompt)
    with output:
        print("Chiffrage de la réforme...")
    api_payload = get_leximpact_payload(openai_response)
    api_response = call_leximpact(api_payload)
    with output:
        print("Rédaction de l'amendement...")
    response_prompt = prepare_response_prompt(openai_response, api_response)
    # Markdown(response_prompt)

    amendement = get_completion(response_prompt, model="gpt-4", temperature=0.5)
    with output:
        print("Rédaction de l'amendement... Terminé")
        display(Markdown(amendement))
    display(Markdown(amendement))
    return Markdown(amendement)

button.on_click(on_button_clicked)

Button(description='Chiffrer et rédiger', style=ButtonStyle())

Output()

In [ ]:
# print("Conversion de la réforme en format OpenFisca...")
# reforme_en_francais = text_input.value
# prompt = get_prompt(reforme_en_francais)
# openai_response = call_openai_function(prompt)
# print("Chiffrage de la réforme...")
# api_payload = get_leximpact_payload(openai_response)
# api_response = call_leximpact(api_payload)
# print("Rédaction de l'amendement...")
# response_prompt = prepare_response_prompt(openai_response, api_response)
# # Markdown(response_prompt)

# amendement = get_completion(response_prompt, model="gpt-4", temperature=0.5)
# display(Markdown(amendement))